# Napari Stitcher batch notebook

With this notebook, you can stitch multiple files, either independently or by concanetating them together. Files are stitched using the Napari Stitcher package. For more information, please read the [documentation](https://multiview-stitcher.github.io/napari-stitcher/main/). Please report any issue through [the project's repository](https://github.com/multiview-stitcher/napari-stitcher).

To run this notebook, just follow the instructions by running the cells in the order. You will first load your data and select your stitching parameters. Then, you will choose between two independent sections depending on whether you want to process your files independently or to concatenate them together. 


In [ ]:
# import packages

import sys
import os, tempfile
import os.path as osp

import numpy as np
from ipyfilechooser import FileChooser
from IPython.display import Markdown
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Label
from pathlib import Path
import xarray as xr
import ngff_zarr
from dask.diagnostics import ProgressBar
from tqdm import tqdm

from multiview_stitcher import (
    registration,
    fusion,
    msi_utils,
    io,
    ngff_utils,
    spatial_image_utils,
)

def printmd(string):
    display(Markdown(string))



## Load data and set stitching parameters

### Get the list of files you want to stitch
All the files need to be in the same folder. You can filter them based on their format and/or name.

In [ ]:
# browse file system to dataset folder
printmd("**Choose a folder containing only the files you want to register**")

fc = FileChooser()
fc.use_dir_icons = True
display(fc)

printmd("**Choose file format**")
format_wid = widgets.Text(value='.czi',placeholder='Type file format',description='File format:')
display(format_wid)

printmd("**Filter files**")
filter_wid = widgets.Text(value='',placeholder='Type in filter',description='Filter by:')
display(filter_wid)

Select the files you want to stitch and define the name of the output filename ending (for files processed independently), or the full name of the output file (for concantenated files).

In [ ]:
# filter files
file_format = format_wid.value
file_filter = filter_wid.value

# get all full path of files in chosen folder
datadir = fc.selected_path  # chosen foder
file_list = os.listdir(datadir)  # list directory
path_list = [osp.join(datadir,fn) for fn in file_list]  # full path list
path_list = [fn for fn in path_list if not osp.isdir(fn)]  # keep only files
if len(file_format) > 0:
    path_list = [fn for fn in path_list if fn.endswith(file_format)]  # filter by file format
if len(file_filter) > 0:
    path_list = [fn for fn in path_list if file_filter in fn]  # filter by file format
path_list.sort()

printmd("**List of files (you can unselect them)**")
file_list_wid = widgets.SelectMultiple(options=path_list,value=path_list,description="File list",layout={'width': 'initial'})
display(file_list_wid)

printmd("**Output filename for concatenated movies or file ending for multiple files**")
out_fn_wid = widgets.Text(value='_stitched.tif',description='Filename:')
display(out_fn_wid)

### Get stitching parameters

In [ ]:
# get registration channel
printmd("**Choose registration channel**")
reg_channel_wid = widgets.BoundedIntText(value=0,min=0,max=10,step=1,description='Channel index:', style={'description_width': 'initial'})
display(reg_channel_wid)

# get pre-registration pruning method
printmd("**Choose pre-registration pruning method**")
pruning_method_wid = widgets.Dropdown(
    options=['none', 'keep_axis_aligned', 'alternating_pattern'],
    value='keep_axis_aligned',
    description='Pruning method:',
    style={'description_width': 'initial'}
)
display(pruning_method_wid)

# get binning factor
printmd("**Choose binning factor**")
binning_wid_x = widgets.IntSlider(value=1,min=1,max=10,step=1,description='Binning factor along X:',style={'description_width': 'initial'})
binning_wid_y = widgets.IntSlider(value=1,min=1,max=10,step=1,description='Binning factor along Y:',style={'description_width': 'initial'})
display(HBox([binning_wid_x, binning_wid_y]))

# get post-registration quality threshold
printmd("**Choose post-registration quality threshold**")
post_reg_quality_bool_wid = widgets.Checkbox(value=True,description='Use post-registration quality threshold',style={'description_width': 'initial'})
post_reg_quality_val_wid = widgets.BoundedFloatText(value=0.2,min=0,max=1.0,step=0.01,description='Quality threshold:', style={'description_width': 'initial'})
display(VBox([post_reg_quality_bool_wid, post_reg_quality_val_wid]))

# get time subset
printmd("**Choose time subset (in frames)**")
time_subset_bool_wid = widgets.Checkbox(value=False,description='Use time subset',style={'description_width': 'initial'})
time_subset_wid_min = widgets.BoundedIntText(value=1,min=1,max=1000,step=1,description='Time subset min:', style={'description_width': 'initial'})
time_subset_wid_max = widgets.BoundedIntText(value=1000,min=1,max=1000,step=1,description='Time subset max:', style={'description_width': 'initial'})
display(VBox([time_subset_bool_wid,HBox([time_subset_wid_min, time_subset_wid_max])]))



In [ ]:
# retrieve stitching parameters
reg_channel_index = reg_channel_wid.value
pruning_method = pruning_method_wid.value if pruning_method_wid.value != 'none' else None
binning = {'x':binning_wid_x.value, 'y':binning_wid_y.value}
post_reg_quality_bool = post_reg_quality_bool_wid.value
post_reg_quality_val = post_reg_quality_val_wid.value
time_subset_bool = time_subset_bool_wid.value
time_subset = (time_subset_wid_min.value, time_subset_wid_max.value)


## Stitch files independently

In [ ]:
# process all files 
for i,fn in enumerate(file_list_wid.value): 
    print("processing {} ({}/{})".format(fn,i+1,len(file_list_wid.value)))

    msims = [msi_utils.get_msim_from_sim(sim, scale_factors=[])
             for sim in io.read_mosaic_image_into_list_of_spatial_xarrays(fn, scene_index=0)]
    
    # filter by time subset
    if time_subset_bool:
        # subset time
        msims = [msi_utils.multiscale_sel_coords(msim,
                {'t': [msi_utils.get_sim_from_msim(msim).coords['t'][it]
                         for it in range(time_subset[0],
                                         time_subset[1] + 1)]})
                  for msim in msims]

    # register data
    temp_dir = tempfile.TemporaryDirectory(dir=datadir)
    with ProgressBar():

        params = registration.register(
            msims,
            transform_key=io.METADATA_TRANSFORM_KEY,
            new_transform_key='affine_registered',
            reg_channel_index=reg_channel_index,
            pre_registration_pruning_method=pruning_method,
            registration_binning=binning,
            post_registration_do_quality_filter=post_reg_quality_bool,
            post_registration_quality_threshold=post_reg_quality_val,
        )

        fused_sim = fusion.fuse(
            [msi_utils.get_sim_from_msim(msim) for msim in msims],
            transform_key='affine_registered',
            #output_chunksize=256,
            )

        fused_ngff_multiscales = ngff_zarr.to_multiscales(
            ngff_utils.sim_to_ngff_image(
                fused_sim,
                transform_key='affine_registered'),
                scale_factors=[])

        output_filename = osp.join(temp_dir.name,'stitched.zarr')
        print(f'Fusing views and saving output to {output_filename}...')
        ngff_zarr.to_ngff_zarr(
            output_filename,
            fused_ngff_multiscales,
            )

        mfused = ngff_utils.ngff_multiscales_to_msim(
            ngff_zarr.from_ngff_zarr(output_filename),
            transform_key='affine_registered')

        output_stitched_fn = fn[:-4]+out_fn_wid.value
        print(f'Streaming into {output_stitched_fn}...')
        io.save_sim_as_tif(
            output_stitched_fn,
            msi_utils.get_sim_from_msim(mfused))

    # delete zarr files
    temp_dir.cleanup()

## Concatenate files together and stitch
First, concatenate files as a zarr array saved in a temporary folder

In [ ]:
# output file
output_tif = osp.join(datadir,out_fn_wid.value)  # type in the output file
print("Output file: ",output_tif)
temp_dir = tempfile.TemporaryDirectory(dir=datadir)
output_filename = Path(osp.join(temp_dir.name,'concat'))

# concatenate files together
print("Concatenating: ",file_list_wid.value)

simss = [io.read_mosaic_image_into_list_of_spatial_xarrays(filename, scene_index=0) for filename in file_list_wid.value]
 
# transform all input files so that they have the same stack properties (spacing, shape, origin) of the first file
simss = [[
    # transformation.transform_sim(
    fusion.fuse(
    [simss[iseq][itile]],
    transform_key=io.METADATA_TRANSFORM_KEY,
    output_stack_properties=spatial_image_utils.get_stack_properties_from_sim(simss[0][itile]))
        for itile in range(len(simss[0]))]
        for iseq in range(len(simss))]

sims = [xr.concat([simss[iseq][itile] for iseq in range(len(simss))], dim='t') for itile in range(len(simss[0]))]
sims = [sim.assign_coords({'t': range(len(sim.coords['t']))}) for sim in sims]

# filter by time subset
if time_subset_bool:
    sims = [spatial_image_utils.sim_sel_coords(sim, {'t': slice(time_subset[0] - 1, time_subset[1] - 1)}) for sim in sims]

print('Convert input into chunked zarr arrays...')
msims = [msi_utils.get_store_decorator(
    output_filename.with_suffix('.tile%03d.zarr' %isim), store_overwrite=True)(
        msi_utils.get_msim_from_sim)(sim, [])
    for isim, sim in tqdm(enumerate(sims), total=len(sims),colour="green",file=sys.stdout,)]

Stitch concatenated data

In [ ]:
# perform registration
# below you can change the registration binning

print('Registering...')
with ProgressBar():
    params = registration.register(
        msims[:],
        transform_key='affine_metadata',
        new_transform_key='affine_registered',
        reg_channel_index=reg_channel_index,
        pre_registration_pruning_method=pruning_method,
        registration_binning=binning,
        post_registration_do_quality_filter=post_reg_quality_bool,
        post_registration_quality_threshold=post_reg_quality_val,
        )

for msim, param in zip(msims, params):
    msi_utils.set_affine_transform(msim, param, transform_key='affine_registered', base_transform_key='affine_metadata')

# perform the fusion
print('Creating fusion graph...')
with ProgressBar():
    fused = fusion.fuse(
        [msi_utils.get_sim_from_msim(msim) for msim in msims],
        transform_key='affine_registered',
        output_chunksize=1024,
        )

print('Creating multiscale output OME-Zarr...')
with ProgressBar():
    mfused = msi_utils.get_msim_from_sim(fused, scale_factors=[])
    mfused.to_zarr(output_filename)

mfused = msi_utils.multiscale_spatial_image_from_zarr(output_filename)

# save fused output as tif file
print('Saving to tif...')
io.save_sim_as_tif(output_tif, mfused['scale0/image'])

Clean temporay data

In [ ]:
# delete zarr files
temp_dir.cleanup()